# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076102 sha256=83952e686a7c1cf9020430cfc70af98a150dd57b3d36a28518189848355161fb
  Stored in directory: c:\users\jacob\appdata\local\pip\cache\wheels\30\bd\87\d0d731ee712c34a4d6bace8b1dbf8c8932e22a1625b4a55be4
Successfully built gmaps


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
csv_file = os.path.join('..', 'WeatherPy', 'Output', 'city_weather.csv')

city_df = pd.read_csv("Output/city_weather.csv")
city_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,338345,Edd,ER,13.9297,41.6925,78.71,80,100,17.31
1,1224061,Wattegama,LK,6.7989,81.4808,77.00,100,40,1.14
2,780687,Berlevåg,NO,70.8578,29.0864,31.68,67,99,12.77
3,3874787,Punta Arenas,CL,-53.1500,-70.9167,41.00,87,40,6.91
4,2155415,New Norfolk,AU,-42.7826,147.0587,61.65,59,0,6.91
...,...,...,...,...,...,...,...,...,...
549,2298330,Medina Estates,GH,5.6823,-0.1641,84.20,84,20,11.50
550,3447928,Seabra,BR,-12.4186,-41.7703,68.00,94,90,3.44
551,2159220,Mackay,AU,-21.1500,149.2000,75.90,73,75,14.97
552,2189343,Kaeo,NZ,-35.1000,173.7833,68.00,56,0,5.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

In [12]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
52,5044290,Saint Anthony,US,45.0205,-93.2180,85.08,18,1,6.91
130,4033077,Vaitape,PF,-16.5167,-151.7500,80.78,67,2,4.03
177,4020109,Atuona,PF,-9.8000,-139.0333,79.61,68,9,6.93
178,100926,Umluj,SA,25.0213,37.2685,80.35,59,0,9.89
186,1255708,Sorbhog,IN,26.5000,90.8667,79.81,43,2,3.02
198,4034551,Faanui,PF,-16.4833,-151.7500,80.78,67,2,3.83
216,1805680,Jingdezhen,CN,29.2947,117.2079,78.96,68,4,4.85
282,1787858,Shangrao,CN,28.4532,117.9686,77.81,67,1,5.99
310,1273109,Devarkonda,IN,16.7000,78.9333,77.77,69,10,5.68
355,2381334,Atar,MR,20.5169,-13.0499,81.82,19,8,4.88


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
52,5044290,Saint Anthony,US,45.0205,-93.2180,85.08,18,1,6.91,
130,4033077,Vaitape,PF,-16.5167,-151.7500,80.78,67,2,4.03,
177,4020109,Atuona,PF,-9.8000,-139.0333,79.61,68,9,6.93,
178,100926,Umluj,SA,25.0213,37.2685,80.35,59,0,9.89,
186,1255708,Sorbhog,IN,26.5000,90.8667,79.81,43,2,3.02,
198,4034551,Faanui,PF,-16.4833,-151.7500,80.78,67,2,3.83,
216,1805680,Jingdezhen,CN,29.2947,117.2079,78.96,68,4,4.85,
282,1787858,Shangrao,CN,28.4532,117.9686,77.81,67,1,5.99,
310,1273109,Devarkonda,IN,16.7000,78.9333,77.77,69,10,5.68,
355,2381334,Atar,MR,20.5169,-13.0499,81.82,19,8,4.88,


In [15]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
52,5044290,Saint Anthony,US,45.0205,-93.2180,85.08,18,1,6.91,Hewing Hotel
130,4033077,Vaitape,PF,-16.5167,-151.7500,80.78,67,2,4.03,InterContinental Bora Bora Resort Thalasso Spa
177,4020109,Atuona,PF,-9.8000,-139.0333,79.61,68,9,6.93,Hôtel Hiva Oa Hanakee Lodge
178,100926,Umluj,SA,25.0213,37.2685,80.35,59,0,9.89,HP Red Sea Hotel
186,1255708,Sorbhog,IN,26.5000,90.8667,79.81,43,2,3.02,NaN
198,4034551,Faanui,PF,-16.4833,-151.7500,80.78,67,2,3.83,InterContinental Bora Bora Resort Thalasso Spa
216,1805680,Jingdezhen,CN,29.2947,117.2079,78.96,68,4,4.85,Fairfield by Marriott Jingdezhen
282,1787858,Shangrao,CN,28.4532,117.9686,77.81,67,1,5.99,Longtan Lake Hotel
310,1273109,Devarkonda,IN,16.7000,78.9333,77.77,69,10,5.68,Meenakshi Hotel
355,2381334,Atar,MR,20.5169,-13.0499,81.82,19,8,4.88,Bab Sahara


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))